In [ ]:
%cd ..

In [ ]:
import json
from pathlib import Path
import sys
import shutil

import numpy as np
import cv2
import pandas as pd

from matplotlib import pyplot as plt

import torch
from torch import nn
from torch.nn import functional as F
from torchvision.io import read_image, write_png
from torchvision.transforms import functional as VF, InterpolationMode

from tqdm import tqdm

from tire_vision.thread import SegmentationInferencer

src_root = Path("data/dataset")
dst_root = Path("data/dataset_crop")

In [ ]:
seg = SegmentationInferencer(device="cuda", target="wheel-tire-thread")

In [4]:
def rembg(image: torch.Tensor) -> torch.Tensor:
    mask = seg(image.to("cuda")).cpu()
    return image * mask + 255 * (1 - mask)


def crop(image: torch.Tensor, padding_frac: float = 0.01) -> torch.Tensor:
    *_, h, w = image.shape
    padding = int(h * padding_frac), int(w * padding_frac)
    mask = seg(image.to("cuda")).cpu()
    image = image * mask + 255 * (1 - mask)
    i, j = torch.where(mask == 1)

    min_i, max_i = torch.min(i), torch.max(i)
    min_j, max_j = torch.min(j), torch.max(j)
    min_i = max(0, min_i - padding[0])
    max_i = min(h, max_i + padding[0])
    min_j = max(0, min_j - padding[1])
    max_j = min(w, max_j + padding[1])

    return image[..., min_i:max_i, min_j:max_j]

In [5]:
if dst_root.exists():
    shutil.rmtree(dst_root)
dst_root.mkdir(parents=True, exist_ok=True)

data = []

for dir in src_root.iterdir():
    label = float(dir.name[:-2].replace(",", "."))
    paths = list(dir.iterdir())
    loop = tqdm(paths, desc=f"Segmenting {label}mm tires")
    for img_path in loop:
        try:
            image = read_image(img_path)
            image = crop(image)

            save_name = f"{len(data)}.png"
            save_path = dst_root / save_name

            data.append([save_name, label])
            write_png(image, save_path)
        except Exception as e:
            print(f"Error: {e}, image: {img_path}")


df = pd.DataFrame(data=data, columns=["path", "label"])
df.to_csv(dst_root / "thread_depths.csv", index=False)

Segmenting 1.8mm tires: 100%|██████████| 2/2 [00:00<00:00,  2.14it/s]
Segmenting 9.4mm tires: 0it [00:00, ?it/s]
Segmenting 2.0mm tires: 100%|██████████| 15/15 [00:07<00:00,  2.04it/s]
Segmenting 9.6mm tires: 0it [00:00, ?it/s]
Segmenting 8.3mm tires:  36%|███▌      | 5/14 [00:01<00:03,  2.76it/s]

Error: min(): Expected reduction dim to be specified for input.numel() == 0. Specify the reduction dim with the 'dim' argument., image: data/dataset/8,3мм/20241204_234902.jpg


Segmenting 3.6mm tires: 100%|██████████| 25/25 [00:12<00:00,  1.93it/s]
Segmenting 9.7mm tires: 0it [00:00, ?it/s]
Segmenting 8.1mm tires: 100%|██████████| 4/4 [00:01<00:00,  2.13it/s]


In [6]:
# clahe = cv2.createCLAHE(clipLimit=5)
# img_path = dst_root / df.sample().iloc[0, 0]

# gray = VF.rgb_to_grayscale(read_image(img_path))[0].numpy()
# grad_x = cv2.Sobel(gray, cv2.CV_16S, 1, 0, ksize=3, scale=1, delta=0, borderType=cv2.BORDER_DEFAULT)
# grad_y = cv2.Sobel(gray, cv2.CV_16S, 0, 1, ksize=3, scale=1, delta=0, borderType=cv2.BORDER_DEFAULT)

# abs_grad_x = cv2.convertScaleAbs(grad_x)
# abs_grad_y = cv2.convertScaleAbs(grad_y)

# grad = cv2.addWeighted(abs_grad_x, 0.5, abs_grad_y, 0.5, 0)
# plt.imshow(grad, cmap="gray")